## Imports

In [86]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df.columns = df.columns.str.replace(' ', '_')
df_test.columns = df_test.columns.str.replace(' ', '_')


## Pipelines

In [97]:
impute_onehot = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
impute_onehot.set_output(transform="pandas")
impute_onehot_cols = ['Style', 'Color', 'Brand', 'Material']

size_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Medium')),
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
]) 
size_pipeline.set_output(transform="pandas")
size_pipeline_cols = ['Size']

yes_no_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
]) 
yes_no_pipeline.set_output(transform="pandas")
yes_no_pipeline_cols = ['Waterproof', 'Laptop_Compartment']

weight_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=18.0)),
    ('scaler', StandardScaler())
])
weight_pipeline.set_output(transform="pandas")
weight_pipeline_cols = ['Weight_Capacity_(kg)']

compartments_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')), 
    ('scaler', StandardScaler())
])
compartments_pipeline.set_output(transform='pandas')
compartments_pipeline_cols = ['Compartments']


## Column Transformer

In [98]:
preprocessor = ColumnTransformer([
    ('style_color_brand_material', impute_onehot, impute_onehot_cols),
    ('size', size_pipeline, size_pipeline_cols),
    ('waterproof_laptopcompartment', yes_no_pipeline, yes_no_pipeline_cols),
    ('weight_pipeline', weight_pipeline, weight_pipeline_cols),
    ('compartments', compartments_pipeline, compartments_pipeline_cols)
])
preprocessor.set_output(transform='pandas')

ColumnTransformer(transformers=[('style_color_brand_material',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='Unknown',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['Style', 'Color', 'Brand', 'Material']),
                                ('size',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='Medium',
                                                                strategy='constant')),
                                                 ('o...
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1))]),
                                 ['Waterproof', 'Laptop_Compartment']),
                                ('weight_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value=18.0,
                                                                strategy='constant')),
                                                 ('scaler', StandardScaler())]),
                                 ['Weight_Capacity_(kg)']),
                                ('compartments',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['Compartments'])])

In [100]:
dfcopy = df.copy()
transformed = preprocessor.fit_transform(dfcopy)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 27 columns):
 #   Column                                            Non-Null Count   Dtype  
---  ------                                            --------------   -----  
 0   style_color_brand_material__Style_Backpack        300000 non-null  float64
 1   style_color_brand_material__Style_Messenger       300000 non-null  float64
 2   style_color_brand_material__Style_Tote            300000 non-null  float64
 3   style_color_brand_material__Style_Unknown         300000 non-null  float64
 4   style_color_brand_material__Color_Black           300000 non-null  float64
 5   style_color_brand_material__Color_Blue            300000 non-null  float64
 6   style_color_brand_material__Color_Gray            300000 non-null  float64
 7   style_color_brand_material__Color_Green           300000 non-null  float64
 8   style_color_brand_material__Color_Pink            300000 non-null  float64
 9   styl